In [1]:
import torch
from torch.autograd import Variable
import torchvision
import torchvision.transforms as T
import random

import numpy as np
from scipy.ndimage.filters import gaussian_filter1d
import matplotlib.pyplot as plt
#from helper.image_utils import SQUEEZENET_MEAN, SQUEEZENET_STD
import helper.image_utils
from PIL import Image

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [2]:
SQUEEZENET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
SQUEEZENET_STD = np.array([0.229, 0.224, 0.225], dtype=np.float32)

In [4]:
from __future__ import print_function

from builtins import range
from six.moves import cPickle as pickle
import numpy as np
import os
from scipy.misc import imread
import platform

def load_imagenet_val(num=None):
    """Load a handful of validation images from ImageNet.

    Inputs:
    - num: Number of images to load (max of 25)

    Returns:
    - X: numpy array with shape [num, 224, 224, 3]
    - y: numpy array of integer image labels, shape [num]
    - class_names: dict mapping integer label to class name
    """
    #imagenet_fn = 'cs231n/datasets/imagenet_val_25.npz'
    imagenet_fn = 'imagenet_val_25.npz'
    if not os.path.isfile(imagenet_fn):
      print('file %s not found' % imagenet_fn)
      print('Run the following:')
      print('cd cs231n/datasets')
      print('bash get_imagenet_val.sh')
      assert False, 'Need to download imagenet_val_25.npz'
    f = np.load(imagenet_fn)
    X = f['X']
    y = f['y']
    class_names = f['label_map'].item()
    if num is not None:
        X = X[:num]
        y = y[:num]
    return X, y, class_names

In [5]:
#from helper.data_utils import load_imagenet_val
#import helper.data_utils as h
X, y, class_names = load_imagenet_val(num=5)

In [3]:
def preprocess(img, size=224):
    transform = T.Compose([
        T.Scale(size),
        T.ToTensor(),
        T.Normalize(mean=SQUEEZENET_MEAN.tolist(),
                    std=SQUEEZENET_STD.tolist()),
        T.Lambda(lambda x: x[None]),
    ])
    return transform(img)

def deprocess(img, should_rescale=True):
    transform = T.Compose([
        T.Lambda(lambda x: x[0]),
        T.Normalize(mean=[0, 0, 0], std=(1.0 / SQUEEZENET_STD).tolist()),
        T.Normalize(mean=(-SQUEEZENET_MEAN).tolist(), std=[1, 1, 1]),
        T.Lambda(rescale) if should_rescale else T.Lambda(lambda x: x),
        T.ToPILImage(),
    ])
    return transform(img)

def rescale(x):
    low, high = x.min(), x.max()
    x_rescaled = (x - low) / (high - low)
    return x_rescaled
    
def blur_image(X, sigma=1):
    X_np = X.cpu().clone().numpy()
    X_np = gaussian_filter1d(X_np, sigma, axis=2)
    X_np = gaussian_filter1d(X_np, sigma, axis=3)
    X.copy_(torch.Tensor(X_np).type_as(X))
    return X

In [4]:
def jitter(X, ox, oy):
    """
    Helper function to randomly jitter an image.
    
    Inputs
    - X: PyTorch Tensor of shape (N, C, H, W)
    - ox, oy: Integers giving number of pixels to jitter along W and H axes
    
    Returns: A new PyTorch Tensor of shape (N, C, H, W)
    """
    if ox != 0:
        left = X[:, :, :, :-ox]
        right = X[:, :, :, -ox:]
        #X = torch.cat([right, left], dim=3)
        X = torch.cat([right, left], 3)
    if oy != 0:
        top = X[:, :, :-oy]
        bottom = X[:, :, -oy:]
        #X = torch.cat([bottom, top], dim=2)
        X = torch.cat([bottom, top], 2)
    return X

In [17]:
def create_class_visualization(target_y, model, dtype, **kwargs):
    """
    Generate an image to maximize the score of target_y under a pretrained model.
    
    Inputs:
    - target_y: Integer in the range [0, 1000) giving the index of the class
    - model: A pretrained CNN that will be used to generate the image
    - dtype: Torch datatype to use for computations
    
    Keyword arguments:
    - l2_reg: Strength of L2 regularization on the image
    - learning_rate: How big of a step to take
    - num_iterations: How many iterations to use
    - blur_every: How often to blur the image as an implicit regularizer
    - max_jitter: How much to gjitter the image as an implicit regularizer
    - show_every: How often to show the intermediate result
    """
    model.type(dtype)
    #l2_reg = kwargs.pop('l2_reg', 1e-3)
    l2_reg = kwargs.pop('l2_reg', 1e-5)
    learning_rate = kwargs.pop('learning_rate', 25)
    #learning_rate = kwargs.pop('learning_rate', 5)
    num_iterations = kwargs.pop('num_iterations', 1000)
    blur_every = kwargs.pop('blur_every', 10)
    max_jitter = kwargs.pop('max_jitter', 16)
    show_every = kwargs.pop('show_every', 25)

    # Randomly initialize the image as a PyTorch Tensor, and also wrap it in
    # a PyTorch Variable.
    img = torch.randn(1, 3, 224, 224).mul_(1.0).type(dtype)
    #img = torch.zeros(1,3,224,224).mul_(1.0).type(dtype)
    img_var = Variable(img, requires_grad=True)

    for t in range(num_iterations):
        # Randomly jitter the image a bit; this gives slightly nicer results
        ox, oy = random.randint(0, max_jitter), random.randint(0, max_jitter)
        img.copy_(jitter(img, ox, oy))

        ########################################################################
        # TODO: Use the model to compute the gradient of the score for the     #
        # class target_y with respect to the pixels of the image, and make a   #
        # gradient step on the image using the learning rate. Don't forget the #
        # L2 regularization term!                                              #
        # Be very careful about the signs of elements in your code.            #
        ########################################################################
        img_var = Variable(img, requires_grad=True)
        
        outputs = model(img_var)
        #print(outputs)
        data_loss = outputs[0][target_y]
        
        reg_loss = l2_reg*torch.norm(img, p=2)
        loss = data_loss - reg_loss
        
        loss.backward()
    
        g = img_var.grad.data 
        #print(g)
        g_norm = torch.norm(g, p=2)
        dx = learning_rate * g /g_norm
        img += dx - l2_reg*img
        
        # zero out gradients before next step
        img_var.grad.data.zero_()
        
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################
        
        # Undo the random jitter
        img.copy_(jitter(img, -ox, -oy))

        # As regularizer, clamp and periodically blur the image
        for c in range(3):
            SQUEEZENET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
            SQUEEZENET_STD = np.array([0.229, 0.224, 0.225], dtype=np.float32)
            
            lo = float(-SQUEEZENET_MEAN[c] / SQUEEZENET_STD[c])
            hi = float((1.0 - SQUEEZENET_MEAN[c]) / SQUEEZENET_STD[c])
            img[:, c].clamp_(min=lo, max=hi)
        if t % blur_every == 0:
            blur_image(img, sigma=0.5)
        
        # Periodically show the image
        if t == 0 or (t + 1) % show_every == 0 or t == num_iterations - 1:
            plt.imshow(deprocess(img.clone().cpu()))
            #class_name = class_names[target_y]
            #plt.title('%s\nIteration %d / %d' % (class_name, t + 1, num_iterations))
            plt.gcf().set_size_inches(6, 6)
            plt.axis('off')
            plt.show()

    return deprocess(img.cpu())

In [6]:
# Download and load the pretrained SqueezeNet model.
#model = torchvision.models.squeezenet1_1(pretrained=True)
#model = torch.load('weightedloss_lrloop.pt')
model = torch.load('weightedloss_lrloop.pt')
model_original = torch.load('weightedloss_lrloop.pt')
model_less1 = torch.load('weightedloss_lrloop.pt')
model_less2 = torch.load('weightedloss_lrloop.pt')
model_less3 = torch.load('weightedloss_lrloop.pt')

# We don't want to train the model, so tell PyTorch not to compute gradients
# with respect to model parameters.
for param in model.parameters():
    param.requires_grad = False

In [32]:
#print(*list(model.children()))
model_less1 = torch.nn.Sequential(*list(model.children())[:-1])
model_less2 = torch.nn.Sequential(*list(model.children())[:-2])
#model_less3.classifier = torch.nn.Sequential(*list(model.classifier.children())[:-7])

In [34]:
print(model_less2)

Sequential (
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (2): ReLU (inplace)
  (3): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (4): Sequential (
    (0): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, 

In [23]:
print(model_original.state_dict())
#print(model.state_dict('fc.weight'))

OrderedDict([('conv1.weight', 
(0 ,0 ,.,.) = 
 -1.0061e-02 -6.5897e-03 -2.4332e-03  ...   5.6166e-02  1.6945e-02 -1.3160e-02
  1.1653e-02  9.2617e-03 -1.1022e-01  ...  -2.7217e-01 -1.3025e-01  2.1235e-03
 -7.7623e-03  5.7535e-02  2.9409e-01  ...   5.1718e-01  2.5342e-01  5.9955e-02
                 ...                   ⋱                   ...                
 -2.9724e-02  1.3485e-02  6.9624e-02  ...  -3.3624e-01 -4.2397e-01 -2.6209e-01
  2.8690e-02  3.8606e-02  6.0251e-02  ...   4.1141e-01  3.9138e-01  1.6326e-01
 -1.4760e-02 -5.5141e-03 -2.5264e-02  ...  -1.5161e-01 -8.2964e-02 -7.0747e-03

(0 ,1 ,.,.) = 
 -9.8029e-03 -2.6102e-02 -3.4324e-02  ...   3.2914e-02  1.1629e-03 -2.5692e-02
  4.7384e-02  3.4191e-02 -1.0401e-01  ...  -3.1277e-01 -1.6119e-01 -2.5284e-03
 -6.6533e-04  9.7556e-02  4.0138e-01  ...   7.0594e-01  3.6639e-01  1.2126e-01
                 ...                   ⋱                   ...                
 -5.7352e-02 -7.3227e-03  2.4566e-02  ...  -4.6479e-01 -5.7397e-01 -3

In [24]:
print(model_original.state_dict()['fc.weight'][2][0])
max_index = 0
max_value = 0
weight_array = [0]*len(model_original.state_dict()['fc.weight'][2])

file = open('weights.txt','w') 

for i in range(len(model_original.state_dict()['fc.weight'][2])):
    file.write(str(i)+",")
    weight_data = model_original.state_dict()['fc.weight'][2][i]
    print(str(i)+","+str(weight_data))
    file.write(str(weight_data)+"\n")
    weight_array[i] = weight_data
    if weight_data > max_value:
        max_index = i
        max_value = weight_data

#file.close() 

print(max_index)
print(max_value)

-0.030020367354154587
0,-0.030020367354154587
1,-0.026974814012646675
2,0.027513599023222923
3,0.04991855099797249
4,-0.040358781814575195
5,0.03069956600666046
6,-0.03415997698903084
7,-0.04099779948592186
8,0.018781524151563644
9,0.03922266140580177
10,0.016461851075291634
11,0.009204909205436707
12,-0.03601802885532379
13,-0.044159479439258575
14,-0.00402899319306016
15,-0.0005820584483444691
16,0.06116460636258125
17,0.05858982354402542
18,-0.0017900713719427586
19,-0.004197895526885986
20,-7.520346844103187e-05
21,0.03581919148564339
22,0.017408020794391632
23,0.016403241083025932
24,-0.02979869581758976
25,0.02184896171092987
26,-0.016505815088748932
27,0.03266691043972969
28,0.013050285167992115
29,0.058307450264692307
30,0.02405647560954094
31,-0.022443154826760292
32,-0.05160761624574661
33,0.03910855948925018
34,0.0460977628827095
35,-0.0471213199198246
36,-0.059711650013923645
37,-0.021314803510904312
38,-0.0078113810159265995
39,-0.004916034173220396
40,0.000537363288458436

In [25]:
#print(weight_array)
print(np.argsort(weight_array)[-5::])

[473 155 205 378 178]


In [119]:
print(model_original.state_dict()['classifier.4.weight'][3338][0])
max_index = 0
max_value = 0
weight_array = [0]*len(model_original.state_dict()['classifier.4.weight'][3338])

file = open('weights.txt','w') 

for i in range(len(model_original.state_dict()['classifier.4.weight'][3338][:])):
    file.write(str(i)+",")
    weight_data = model_original.state_dict()['classifier.4.weight'][3338][i]
    print(str(i)+","+str(weight_data))
    file.write(str(weight_data)+"\n")
    weight_array[i] = weight_data
    if weight_data > max_value:
        max_index = i
        max_value = weight_data

#file.close() 

print(max_index)
print(max_value)

0.012135487049818039
0,0.012135487049818039
1,-0.00800168327987194
2,-0.003159527899697423
3,0.012174329720437527
4,0.0031203618273139
5,-0.00039207725785672665
6,-0.00881992932409048
7,-0.007245725020766258
8,-0.0020440355874598026
9,0.00011859529331559315
10,-0.01677616685628891
11,0.005398652516305447
12,-0.015016579069197178
13,0.012648220174014568
14,-0.011400511488318443
15,-0.007508728187531233
16,-0.01302038412541151
17,0.006648382171988487
18,0.00747642619535327
19,0.011222966946661472
20,0.00022921650088392198
21,-0.013325504027307034
22,0.019620288163423538
23,-0.014083849266171455
24,0.0006634350284002721
25,0.0016500671626999974
26,0.0012730928137898445
27,0.002823163755238056
28,0.007249427028000355
29,-0.0014772224240005016
30,-0.0035533462651073933
31,-0.0006784379365853965
32,-0.007185627706348896
33,-0.014446771703660488
34,0.009787967428565025
35,-0.01026867050677538
36,-0.00712290033698082
37,-0.0076102144084870815
38,0.0009575397125445306
39,-0.009051725268363953
4

In [120]:
#print(weight_array)
print(np.argsort(weight_array)[-5::])

[1414 3735 3925 2272  713]


In [161]:
#print(model_original.state_dict()['classifier.6.weight'][:][500])
max_index = 0
max_value = 0


for i in range(5):
    file.write(str(i)+",")
    weight_data = model_original.state_dict()['classifier.6.weight'][i][7]
    bias_data = model_original.state_dict()['classifier.6.bias'][i]
    print(str(i)+","+str(weight_data+bias_data))
    if (weight_data+bias_data) > max_value:
        max_index = i
        max_value = weight_data + bias_data

#file.close() 

print(max_index)
print(max_value)

0,-0.01397975703002885
1,0.0031366413459181786
2,-0.011282742605544627
3,-0.03238751366734505
4,0.0227012038230896
4
0.0227012038230896


In [35]:
dtype = torch.FloatTensor
dtype = torch.cuda.FloatTensor # Uncomment this to use GPU
model.type(dtype)

target_y = 205
#target_y = 76 # Tarantula
# target_y = 78 # Tick
# target_y = 187 # Yorkshire Terrier
# target_y = 683 # Oboe
# target_y = 366 # Gorilla
# target_y = 604 # Hourglass
out = create_class_visualization(target_y, model_less2, dtype)

RuntimeError: backward should be called only on a scalar (i.e. 1-element tensor) or with gradient w.r.t. the variable